# Art Blocks Mint Data

In this NB, I'm assembling and exploring sample data on minting for Art Blocks NFTs.

## Data ingestion

I'm pulling in data from BigQuery's public data tables on Ethereum transactions and logs:
- `bigquery-public-data.crypto_ethereum.transactions`
- `bigquery-public-data.crypto_ethereum.logs`

We're using these tables to analyze mint transactions made against the Art Blocks contract, and then viewing associated logs, which contain identifiers for the specific NFT being minted, in another contract:
- Minting contract: `0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7`
- Logs contract: `0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270`


In [1]:
# If you want to run this you'll have to ask Shane how to get a Google service key. Alternatively, use the provided CSV.
%env GOOGLE_APPLICATION_CREDENTIALS = C:\Users\shane\analytics\ethereum\service_key.json
from google.cloud import bigquery
client = bigquery.Client()
import matplotlib.pyplot as plt
import pandas as pd

sql = """
with

transactions as (
SELECT *
FROM `bigquery-public-data.crypto_ethereum.transactions` as t
WHERE to_address = '0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7'
),

logs as (
SELECT transaction_hash,
MAX(CAST(ARRAY(SELECT * from l.topics)[ordinal(4)] AS INT)) as raw_id
FROM `bigquery-public-data.crypto_ethereum.logs` as l
WHERE address = '0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
AND ARRAY_LENGTH(ARRAY(SELECT * from l.topics)) = 4
GROUP BY transaction_hash
)

SELECT 
t.*,
l.raw_id as raw_id
from transactions as t 
join logs as l 
on t.hash = l.transaction_hash
ORDER BY t.block_number desc
"""
df_raw = client.query(sql).to_dataframe()

env: GOOGLE_APPLICATION_CREDENTIALS=C:\Users\shane\analytics\ethereum\service_key.json


## Data curation

To make the data easier to work with, I do a few cleaning tasks:
1. Process the `raw_id` from the logs into a `project_id` and `item_id`.
2. Convert the `value` of the transaction in wei to `mint_price_eth` in eth.
3. _Let Shane know if you want to see more cleaning..._

In [2]:
df = df_raw

# Split the raw ID into the project ID and the item ID
def retrieveID (raw_id, type):
    raw_id_str = str(raw_id)
    if len(raw_id_str) == 7:
        project_id_str = raw_id_str[:1]
        item_id_str = raw_id_str[1:]
    elif len(raw_id_str) == 8:
        project_id_str = raw_id_str[:2]
        item_id_str = raw_id_str[2:]
    elif len(raw_id_str) == 9:
        project_id_str = raw_id_str[:3]
        item_id_str = raw_id_str[3:]
    else:
        raise ValueError("raw_id length not between 7 and 9 characters")

    project_id = int(project_id_str)
    item_id = int(item_id_str)
    if type == 'project':
        return project_id
    elif type == 'item':
        return item_id
    else:
        raise ValueError("please choose type from ['project','item']")

df['project_id'] = df['raw_id'].apply(lambda x: retrieveID(x,'project'))
df['item_id'] = df['raw_id'].apply(lambda x: retrieveID(x,'item'))
df['mint_price_eth'] = df['value'] / 1000000000000000000

## Data exploration

### Understanding our tables

Here are the variables we now have available:

In [3]:
df.dtypes


hash                                        object
nonce                                        int64
transaction_index                            int64
from_address                                object
to_address                                  object
value                                       object
gas                                          int64
gas_price                                    int64
input                                       object
receipt_cumulative_gas_used                  int64
receipt_gas_used                             int64
receipt_contract_address                    object
receipt_root                                object
receipt_status                               int64
block_timestamp                datetime64[ns, UTC]
block_number                                 int64
block_hash                                  object
max_fee_per_gas                            float64
max_priority_fee_per_gas                   float64
transaction_type               

And the head:

In [4]:
df.head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,raw_id,project_id,item_id,mint_price_eth
0,0xecc14d8d5570db1b03b321a011a671596b82eac058f8...,913,218,0x80b1960ce559fdf3f7543b0d87fbb5381f8c3903,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,0E-9,500000,63476764347,0xefef39a1000000000000000000000000000000000000...,19011630,...,14630456,0x75cbbeb06d53a01d9c06ba514516e72cb8a4a6b5ade7...,6.688012e+10,1.500000e+09,2.0,63476764347,294000025,294,25,0E-9
1,0x177c1202e9e42d1eeeb6e21f044cf5cf7230e00121d7...,10,166,0xbc3b66d61e6df21a61acc2a331ddf004601af91d,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,0E-9,500000,69903622607,0xefef39a1000000000000000000000000000000000000...,19256840,...,14630449,0xa0f5f2b184d0b7dec25e1261d2361b2938d8bce90ab5...,7.504527e+10,1.500000e+09,2.0,69903622607,294000024,294,24,0E-9
2,0xc407a2d803db6981c9115f4b81e66cd6dc6f637b0c6d...,831,224,0xeeaf61935c0c48f8f8cc28c381683d7fa2f75e1c,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,0E-9,500000,90216853461,0xefef39a1000000000000000000000000000000000000...,16201682,...,14630397,0x758f127b32b964d667e07f1ec7ee11b2f78c04352493...,9.752802e+10,1.500000e+09,2.0,90216853461,294000023,294,23,0E-9
3,0x525c671dd6dcdf1cede69ef8e2c65b49708d69461ce4...,49,152,0x7f890833ebecb57493ea7a2345143ed67e6f2d9b,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,50000000000000000.000000000,500000,72182869364,0xefef39a1000000000000000000000000000000000000...,18950521,...,14630307,0x1e95435b6a9a585b247dcbbb9421743c0d934973e4c1...,7.576315e+10,1.500000e+09,2.0,72182869364,263000657,263,657,0.050000000
4,0xfd24e14b192c23d948704f0d4f63454d4de44b35cac4...,7353,82,0x0f0eae91990140c560d4156db4f00c854dc8f09e,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,0E-9,500000,61404114958,0xefef39a1000000000000000000000000000000000000...,5880048,...,14630257,0xb9d2d31e01113ac58b871bbe79bdcb870927b010e541...,9.685228e+10,2.000000e+09,2.0,61404114958,294000022,294,22,0E-9


### Exploring collections

An obvious place to start exploring the data would be to count the mints grouping by the project:

In [5]:
items_per_project = df.groupby(['project_id']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
items_per_project

,project_id,counts
13,116,6158
14,117,4712
95,203,2048
119,227,2048
174,283,1999
...,...,...
185,294,26
6,57,1
186,295,1
188,298,1


We can do a quick manual validation with the Art Blocks website:
1. [https://www.artblocks.io/project/116](https://www.artblocks.io/project/116) confirms that Project 116 had 6158 items minted. &#9745;
2. [https://www.artblocks.io/project/117](https://www.artblocks.io/project/117) confirms that Project 117 had 4712 items minted. &#9745;
3. [https://www.artblocks.io/project/283](https://www.artblocks.io/project/283) suggests Project 283 has 2000 items minted but we only have 1999. &#9746;
    - Could be that the project creator minted themselves a single item beforehand through some mechanism other than the standard? You can run `set(range(2000)) - set(df.loc[df['project_id'] == 283]['item_id'])` to see that the missing `item_id` from our records is `0`.
4. [https://www.artblocks.io/project/57](https://www.artblocks.io/project/57) suggests Project 57 has 334 items minted but we only have 1. &#9746;
    - The minting contract we're using was deployed in July 2021, whereas this project minted in May 2021, primarily on a previous contract.
5. [https://www.artblocks.io/project/300](https://www.artblocks.io/project/300) suggests Project 300 has 1 item minted (so far) &#9745;



We might also want to know the total mint proceeds by collection:

In [6]:
proceeds_per_project = df.groupby(['project_id'])['mint_price_eth'].sum().reset_index(name='proceeds').sort_values(by=['proceeds'], ascending=False)
proceeds_per_project

,project_id,proceeds
57,163,6170.750000000
67,173,5377.150000000
34,138,4995.250000000
41,145,4079.710000000
43,147,3248.100100000
...,...,...
120,228,0E-9
81,189,0E-9
100,208,0E-9
185,294,0E-9


[Meridian by Matt DesLauriers](https://www.artblocks.io/project/163) raised over 6000 ETH (18MM USD as of this writing) from minting. Wow! They are very beautiful, to be fair.

Maybe we also want to get a sense of which projects minted the most quickly. This is a little complex and subjective:
- Some projects have many items whereas others have few. And they obviously have different prices.
- Different projects have different pricing structures. A dutch auction may play out more slowly than a flat pricing strategy, particularly if the latter is priced below market level, in which case the full mint may occur almost instantaneously upon unlock.
- Comparing the timestamp of first mint to last could be misleading because sometime creators mint items for themselves or friends/family before the public mint. Although it's available on Art Blocks website, we don't have the timestamp for publicly accesible minting explicitly in our data.

But, just for fun, let's define and compute three quick metrics:
1. **Max item velocity (MIV) per minute**: how many units were minted in the project's hottest minute?
2. **Max value velocity (MVV) per minute**: what were the mint proceeds in the project's hottest minute?
3. **Max user velocity (MUV) per minute**: how many distinct addresses minted in the project's hottest minute?


In [19]:

df['minute'] = df['block_timestamp'].dt.floor('min')

# MIV
project_minute_items = df.groupby(['project_id','minute']).size().reset_index(name='counts')
miv = project_minute_items.groupby(['project_id']).max().sort_values(by=['counts'], ascending=False)
miv = miv.rename(columns={"minute":"miv_minute","counts":"miv"})

# MVV
project_minute_value = df.groupby(['project_id','minute'])['mint_price_eth'].sum().reset_index(name='counts')
mvv = project_minute_value.groupby(['project_id']).max().sort_values(by=['counts'], ascending=False)
mvv = mvv.rename(columns={"minute":"mvv_minute","counts":"mvv"})

# MUV
project_minute_users = df.groupby(['project_id','minute'])['from_address'].nunique().reset_index(name='counts')
muv = project_minute_users.groupby(['project_id']).max().sort_values(by=['counts'], ascending=False)
muv = muv.rename(columns={"minute":"muv_minute","counts":"muv"})

combined_project_scores = miv.join(muv).join(mvv)

In [20]:
combined_project_scores

,miv_minute,miv,muv_minute,muv,mvv_minute,mvv
project_id,,,,,,
141,2021-08-23 19:06:00+00:00,610,2021-08-23 19:06:00+00:00,610,2021-08-23 19:06:00+00:00,1133.000000000
232,2021-12-20 18:32:00+00:00,593,2021-12-20 18:32:00+00:00,573,2021-12-20 18:32:00+00:00,593.000000000
203,2021-11-16 22:11:00+00:00,569,2021-11-16 22:11:00+00:00,493,2021-11-16 22:11:00+00:00,85.350000000
138,2021-08-27 16:02:00+00:00,527,2021-08-27 16:02:00+00:00,527,2021-08-27 16:02:00+00:00,2635.000000000
206,2021-11-15 18:29:00+00:00,418,2021-11-15 18:29:00+00:00,370,2021-11-15 18:29:00+00:00,837.200000000
...,...,...,...,...,...,...
234,2021-12-15 19:52:00+00:00,1,2021-12-15 19:52:00+00:00,1,2021-12-15 19:52:00+00:00,2.000000000
294,2022-04-21 20:35:00+00:00,1,2022-04-21 20:35:00+00:00,1,2022-04-21 20:35:00+00:00,0E-9
295,2022-04-12 02:48:00+00:00,1,2022-04-12 02:48:00+00:00,1,2022-04-12 02:48:00+00:00,0.050000000


MIV and MUV winner: [Rapture by Thomas Lin Pederson](https://www.artblocks.io/project/141) minted 610 units (to 610 unique addresses) in a single minute!

MVV winner: [Geometry Runners by Rich Lord](https://www.artblocks.io/project/138) generated 2635 ETH (on 527 mints) in a single minute!

They're cool, I guess, but if you don't like [Meridian by Matt DesLauriers](https://www.artblocks.io/project/163) better then my scientific opinion is that you have poor taste.

### Exploring items

Won't spend too much time here but let's see the highest price anybody has paid to mint:

In [9]:
df.sort_values(by=['mint_price_eth'], ascending=False).head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,raw_id,project_id,item_id,mint_price_eth,minute
60078,0x746c6f0428c8ce0eb9369d02237c37df27612cae454d...,103,69,0x3d0b4b2083c87f06507cefe924ecebe56f21c5ab,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,15000000000000000000.000000000,601089,75500750052,0xefef39a1000000000000000000000000000000000000...,4260892,...,0x311df8d2f2be07fbbdffd54cadb767466801afec3664...,7.550075e+10,7.550075e+10,2.0,75500750052,173000001,173,1,15.000000000,2021-09-27 16:03:00+00:00
75470,0xa058cfa934ca030222b593f5dce856aa167f79372522...,108,96,0x70d00fc2a472ea9149952186dc264dfef421b37f,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,15000000000000000000.000000000,599215,234933110184,0xefef39a1000000000000000000000000000000000000...,5973128,...,0xfcb6cba1a04afa84ddd9f5c0d22fc1f9181cf16ab2bc...,2.349331e+11,2.349331e+11,2.0,234933110184,144000001,144,1,15.000000000,2021-08-30 19:05:00+00:00
73172,0x57c87558770016d9900084b67260fd8a8946aa409659...,746,5,0xda15b2fd4eb7967039c7e46e69de77fee2e235df,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,15000000000000000000.000000000,573619,383661218929,0xefef39a1000000000000000000000000000000000000...,665066,...,0x54295a4d7f67461b820603468ac43b07b27b539c7157...,1.000000e+12,1.958561e+11,2.0,383661218929,147000001,147,1,15.000000000,2021-09-03 16:02:00+00:00
73171,0x3a205154ca1fbd3f68a44758d45ee8f1a59ba04e37ec...,1256,21,0xf3e6ba583b960f7a8b395212fdce8a92a0df912c,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,15000000000000000000.000000000,596725,297351301115,0xefef39a1000000000000000000000000000000000000...,2034106,...,0x54295a4d7f67461b820603468ac43b07b27b539c7157...,NaN,NaN,NaN,297351301115,147000002,147,2,15.000000000,2021-09-03 16:02:00+00:00
73170,0x0db1f43cd5976b11e965678b72a57a0380910d430081...,289,344,0xbed872177df6a565b7190873f9540d8ca224f607,0x47e312d99c09ce61a866c83cbbbbed5a4b9d33e7,15000000000000000000.000000000,580423,195856066589,0xefef39a1000000000000000000000000000000000000...,25078160,...,0x229108aa6bcbae558ecf371ec4ed3b5f8e9648a5e386...,1.958561e+11,1.958561e+11,2.0,195856066589,147000003,147,3,15.000000000,2021-09-03 16:04:00+00:00


People have paid up to 15ETH for early items in Projects [147](https://www.artblocks.io/project/147) and [163](https://www.artblocks.io/project/163). These aren't necessarily the highest valued items within those collections. Rather, it's probably just the first few mints in collections using dutch auctions. To get at the items that are actually valued the highest, you'd be more interested in subsequent transactions after the mints. But given that you never know if the buyer, or someone making an offer, is actually someone other than the seller, the notion of an item's true market value is pretty murky.

### Exploring users

We may want to combine this minting data with other data on the activity of users. For instance, Nansen categorizes some users as Smart Traders and/or Smart HODLers, among other categories, based on their minting and trading histories and then allows their subscribers to track activities specific to those groups. Going deep into user performance is out of scope for this exercise, but let's define and measure a few simple measures:
1. **Experience score (ES)**: a counter of how many different projects each user has minted in.
2. **Investment score (IS)**: the user's total ETH spend on minting Art Blocks, not including gas.
3. **Popularity score (PS)**: we rank project popularity based on total mint proceeds and then a user's popularity score is the average over the popularity scores of their mints. Lower scores indicate the user has preferred more popular projects.

In [25]:
print("We had", len(df), "mints across", len(df.from_address.unique()), "unique minters.")

e_s =  df.groupby(['from_address'])['project_id'].nunique().reset_index(name='counts')
e_s = e_s.rename(columns={"from_address":"user","counts":"es"})

i_s =  df.groupby(['from_address'])['mint_price_eth'].sum().reset_index(name='counts')
i_s = i_s.rename(columns={"from_address":"user","counts":"is"})

project_popularity = df.groupby(['project_id'])['mint_price_eth'].sum().reset_index(name='counts')
project_popularity['rank'] = project_popularity['counts'].rank(method='min',ascending=False)
project_popularity = project_popularity.rename(columns={"rank":"project_pop_rank","counts":"project_pop_score"})
df_with_pop = df.merge(project_popularity,on='project_id', how='left')
p_s = df_with_pop.groupby(['from_address'])['project_pop_rank'].mean().reset_index(name='counts')
p_s = p_s.rename(columns={"from_address":"user","counts":"ps"})

combined_user_scores = e_s.merge(i_s,on='user', how='left').merge(p_s,on='user', how='left')



We had 113979 mints across 29395 unique minters.


In [27]:
combined_user_scores.head()

,user,es,is,ps
0,0x000000000004d7463d0f9c77383600bc82d612f5,1,0.150000000,93.000000
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2,0.500000000,64.500000
2,0x0000000000a47f168017e6624f59488924ec6f50,2,0.105000000,115.500000
3,0x000000000da41b18fa6b9ba31d835335cd34b8c4,1,0.100000000,86.000000
4,0x000000000f4141775240685a4e005ed819846840,3,4.750000000,23.333333


Rather than doing any additional analysis with this data, I'm just going to export it as its own CSV such that we can consider incorporating filters based on it into the visualization of the mint data.

## Data export

In [28]:
df.to_csv('artblocks_mint_data.csv')
combined_user_scores.to_csv('artblocks_user_scores.csv')

## Useful links

1. To search for project _k_, go to `artblocks.io/project/k`. [Example](https://www.artblocks.io/project/182).
2. To search for an item, find its `raw_id` above and go to `artblocks.io/token/raw_id`. [Example](https://www.artblocks.io/token/182000147).
    - You can see it on OpenSea at `https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/raw_id`. [Example](https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/182000147)
    - You can see it on Etherscan at `https://etherscan.io/token/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270?a=raw_id`. [Example](https://etherscan.io/token/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270?a=182000147)
    - You can get a png version at `https://artblocks-mainnet.s3.amazonaws.com/raw_id.png`. [Example](https://artblocks-mainnet.s3.amazonaws.com/182000147.png)
    - You can a fully dynamic/interactive version at `https://generator.artblocks.io/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/raw_id`. [Example](https://generator.artblocks.io/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/182000147)
